In [4]:
pip install gdown 

Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install pyarrow

     |████████████████████████████████| 42.3 MB 12.8 MB/s eta 0:00:01


In [15]:
pip install fastparquet

     |████████████████████████████████| 1.7 MB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 83.2 MB/s eta 0:00:01
     |████████████████████████████████| 13.1 MB 80.8 MB/s eta 0:00:01
     |████████████████████████████████| 19.5 MB 89.1 MB/s eta 0:00:01
     |████████████████████████████████| 347 kB 97.9 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.2
    Uninstalling pandas-1.4.2:
      Successfully uninstalled pandas-1.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 2.0.2 which is incompatible.
numba 0.55.1 requ

In [ ]:
pip install mlflow boto3 

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,train_test_split
from sklearn.metrics import roc_auc_score,roc_curve,confusion_matrix,auc
import xgboost as xgb

In [7]:
import os
import gdown
def download_file_from_gdrive(file_id,destination_path):
    if not os.path.exists(destination_path):
        print(f"Downloading file from Google Drive with ID: {file_id}")
        url = f"https://drive.google.com/uc?id={file_id}"
        gdown.download(url,destination_path,quiet=False)
        print(f"Download complete: {destination_path}")
    else:
        print(f"Skipping download.")
file_id = '1OP0kWJWGldkxXCWdwoEEhup4FTPn0isj'  #  file's actual google link ID
destination_path = '/workspaces/Real-Time-Fraud-Detection/data_cleaned/train_data.parquet'  # Path to where the file should be saved

download_file_from_gdrive(file_id, destination_path)
            

Downloading...
From: https://drive.google.com/uc?id=1OP0kWJWGldkxXCWdwoEEhup4FTPn0isj
To: /workspaces/Real-Time-Fraud-Detection/data_cleaned/train_data.parquet
100%|███████████████████████████████████████████████████| 84.8M/84.8M [00:00<00:00, 135MB/s]

Download complete: /workspaces/Real-Time-Fraud-Detection/data_cleaned/train_data.parquet


In [27]:
pip install --upgrade pandas

Note: you may need to restart the kernel to use updated packages.


In [29]:
pd.__version__

'1.4.2'

In [ ]:
import boto3
print(boto3.__version__)

In [ ]:
import os

# Set the AWS profile to use
os.environ['AWS_PROFILE'] = 'ml_user'

In [ ]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Fraud detection")

In [25]:
import dask.dataframe as dd
df=dd.read_parquet("/workspaces/Real-Time-Fraud-Detection/data_cleaned/train_data.parquet")
print(df.head())

   TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400        4.226834         W  13926   
1        2987001        0          86401        3.367296         W   2755   
2        2987002        0          86469        4.077537         W   4663   
3        2987003        0          86499        3.912023         W  18132   
4        2987004        0          86506        3.912023         H   4497   

   card2  card3       card4  card5  ... id_35  id_36  id_37  id_38 DeviceType  \
0    NaN  150.0    discover  142.0  ...  None   None   None   None       None   
1  404.0  150.0  mastercard  102.0  ...  None   None   None   None       None   
2  490.0  150.0        visa  166.0  ...  None   None   None   None       None   
3  567.0  150.0  mastercard  117.0  ...  None   None   None   None       None   
4  514.0  150.0  mastercard  102.0  ...     T      F      T      T     mobile   

                      DeviceInfo                Da

In [3]:
def preprocess_numerical_features(df,drop_col=None):
    
    START_DATE='2017-12-01'
    startdate=datetime.datetime.strptime(START_DATE,"%Y-%m-%d")
    #convert transactional date into actual date
    df["Date"]=df['TransactionDT'].apply(lambda x:(startdate + datetime.timedelta(seconds=x)))
    #extract features
    df['weekdays']=df['Date'].dt.dayofweek
    df['Hour']=df['Date'].dt.hour
    df['Day']=df['Date'].dt.day
    
    #log transform of transaction amount-to redice skew
    df['TransactionAmt']= np.log(df['TransactionAmt'])
    
    #drop transactional date column
    
    if drop_cols:
        df.drop(drop_cols,axis=1,inplace=True)
    
    return df

In [ ]:
drop_col=['TransactionDT']
df_train=preprocess_numerical_features(df_train,drop_col=drop_col)
df_test=preprocess_numerical_features(df_test,drop_col=drop_col)

In [ ]:
def encode_categorical_features(train_df,test_df):
    train_df=train_df.copy
    test_df=test_df.copy
    
    for col in train_df.columns:
        if train_df[col].dtype == 'object':
            #fill nan in both
            train_df[col] = train_df[col].fillna('Unknown')
            test_df[col] = test_df[col].fillna('Unknown')
            
            #fit label encoder
            le=LabelEncoder()
            train_df['col']=le.fit_transform(train_Df[col].astype(str))
            test_df['col']=le.fit_transform(test_Df[col].astype(str))
            
    return train_df,test_df    

In [ ]:
train,test=encode_categorical_features(df_train,df_test)

In [ ]:
X=train.drop('isFraud',axis=1)
y=train['isFraud']
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=0)


In [ ]:
#calculate class weight for imbalance data
scale_pos_weight =  (y == 0).sum() / (y == 1).sum()

In [ ]:
with mlflow.start_run():
    params=dict(n_estimators=500,
                max_depth=9,
                learning_rate=0.05,
                subsample=0.9,
                colsample_bytree=0.9,
                tree_method='gpu_hist')
    mlflow.log_params(params)
    model=xgb.XGBClassifier(
        **params,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        eval_metric='logloss',
        missing=np.nan)
    model.fit(X_train,y_train)
    y_pred = model.predict(X_val)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
        
    auc_score = roc_auc_score(y_val, y_pred_proba)
    print(f" Model trained! AUC Score: {auc_score:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Log the model to MLflow
    mlflow.xgboost.log_model(model, artifact_path="model")
        